# Hosting Strands Multi-Agent System on Amazon Bedrock AgentCore Runtime with MCP Integration

This notebook demonstrates how to deploy a Strands multi-agent mortgage assistant system to Amazon Bedrock AgentCore Runtime while connecting to MCP tools hosted on another AgentCore Runtime.

## Overview

### Tutorial Details

| Information         | Details                                                   |
|:--------------------|:----------------------------------------------------------|
| Tutorial type       | Multi-Agent System Deployment                             |
| Agent type          | Multi-Agent with MCP Integration                          |
| Agentic Framework   | Strands Agents                                            |
| LLM model           | Anthropic Claude Haiku                                    |
| Tutorial components | Hosting Strands Agents on AgentCore Runtime + MCP Client |
| Tutorial vertical   | Mortgage/Financial Services                               |
| Example complexity  | Advanced                                                  |
| SDK used            | Amazon BedrockAgentCore Python SDK, MCP, and Strands     |

### Tutorial Architecture

```
Client → AgentCore Runtime (Strands Multi-Agent) → AgentCore Runtime (MCP Server)
                    ↓
            Knowledge Base (Bedrock)
```

### Prerequisites

1. Completed MCP server deployment in `../01-mcp-server-hosting/`
2. Created Knowledge Base from `../../2_bedrock-multi-agent/01_create_knowledgebase.ipynb`

## 1. Setup and Installation

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## 2. Load Configuration from MCP Server Deployment

In [ ]:
# Load MCP server configuration from the previous deployment
%store -r mcp_url
%store -r cognito_config
%store -r kb_id

print(f"MCP Server Endpoint: {mcp_url}")
print(f"Knowledge Base ID: {kb_id}")
print(f"Cognito Config: {cognito_config}")

## 3. Create Strands Multi-Agent Application for AgentCore Runtime

Create the main agent file that will be deployed to AgentCore Runtime:

In [ ]:
%%writefile strands_multi_agent.py
import os
import json
from strands import Agent, tool
from strands_tools import retrieve, calculator
from strands.tools.mcp import MCPClient
from datetime import timedelta
from mcp.client.streamable_http import streamablehttp_client
from bedrock_agentcore.runtime import BedrockAgentCoreApp
import boto3
from boto3.session import Session

app = BedrockAgentCoreApp()

# Configuration from environment variables
MCP_URL = os.environ.get('MCP_URL')
COGNITO_CLIENT_ID = os.environ.get('COGNITO_CLIENT_ID')
COGNITO_USER_POOL_ID = os.environ.get('COGNITO_USER_POOL_ID')
COGNITO_USERNAME = os.environ.get('COGNITO_USERNAME')
COGNITO_PASSWORD = os.environ.get('COGNITO_PASSWORD')
KNOWLEDGE_BASE_ID = os.environ.get('KNOWLEDGE_BASE_ID')


def get_cognito_token():
    """Get Cognito authentication token."""
    boto_session = Session()
    region = boto_session.region_name
    # Initialize Cognito client
    cognito_client = boto3.client('cognito-idp', region_name=region)
    # Authenticate User and get Access Token
    response = cognito_client.initiate_auth(
        ClientId=COGNITO_CLIENT_ID,
        AuthFlow='USER_PASSWORD_AUTH',
        AuthParameters={
            'USERNAME': COGNITO_USERNAME,
            'PASSWORD': COGNITO_PASSWORD
        }
    )
    return response['AuthenticationResult']['AccessToken']

@tool
def answer_general_mortgage_questions(query):
    """Answer general mortgage questions using knowledge base."""
    general_mortgage_agent = Agent(
        model="us.anthropic.claude-3-5-haiku-20241022-v1:0",
        tools=[retrieve],
        system_prompt="""
        You are a mortgage bot, and can answer questions about mortgage refinancing and tradeoffs of mortgage types. Greet the customer first.
        
        IMPORTANT: Always use the retrieve tool to search the knowledge base before answering any mortgage-related questions.
        
        You can:
        1. Provide general information about mortgages
        2. Handle conversations about general mortgage questions, like high level concepts of refinancing or tradeoffs of 15-year vs 30-year terms.
        3. Offer guidance on the mortgage refinancing and tradeoffs of mortgage types.
        4. Access a knowledge base of mortgage information using the retrieve tool
        5. Only answer from the knowledge base and not from your general knowledge. If you dont have the answer from Knowledge base, say "I dont know"
        
        When helping users:
        - ALWAYS call the retrieve tool first to search for relevant information
        - Provide clear explanations based on retrieved information
        - Use plain language to explain complex financial terms
        - Offer balanced advice considering both pros and cons
        - Be informative without making specific financial recommendations
        
        Remember that you're providing general mortgage information, not financial advice.
        Always clarify that users should consult with a financial advisor for personalized advice.
        """
    )

    return str(general_mortgage_agent(query))

@tool
def get_mortgage_details(customer_id):
    """Get mortgage details for a customer."""
     # TODO: Implement real business logic to retrieve mortgage status
    return {
        "account_number": customer_id,
        "outstanding_principal": 150000.0,
        "interest_rate": 4.5,
        "maturity_date": "2030-06-30",
        "payments_remaining": 72,
        "next_payment_due": "2024-07-01",
        "next_payment_amount": 1250.0
    }

@tool
def answer_existing_mortgage_questions(query):
    """Answer questions about existing mortgages."""
    existing_mortgage_agent = Agent(
        model="us.anthropic.claude-3-5-haiku-20241022-v1:0",
        tools=[get_mortgage_details],
        system_prompt="""
        You are an Existing Mortgage Assistant that helps customers with their current mortgages.

        You can:
        1. Provide information about a customer's existing mortgage
        2. Check mortgage status including balance and payment information
        3. Evaluate refinancing eligibility
        4. Calculate payoff timelines with extra payments
        5. Answer questions about mortgage terms and conditions

        When helping users:
        - Always verify the customer ID before providing information
        - Provide clear explanations of mortgage details
        - Format financial data in a readable way
        - Explain payment schedules and upcoming due dates
        - Offer guidance on refinancing options when appropriate
        - Use the knowledge base for detailed information when needed

        Remember that you're dealing with sensitive financial information, so maintain a professional tone
        and ensure accuracy in all responses.
        """
    )
    return str(existing_mortgage_agent(query))

@app.entrypoint
def mortgage_assistant(payload):
    """
    Multi-agent mortgage assistant with MCP integration.
    """
    user_input = payload.get("prompt")
    print(f"Processing query: {user_input}")
    
    try:
        # Get authentication token for MCP server
        token = get_cognito_token()
        
        # Create MCP client for AgentCore Runtime
        mcp_client = MCPClient(lambda: streamablehttp_client(
            url=MCP_URL,
            headers={
                "Authorization": f"Bearer {token}",
                "Content-Type": "application/json"
            },
            timeout=timedelta(seconds=120)
        ))
        
        # Use MCP client context manager
        with mcp_client:
            # Get MCP tools from AgentCore Runtime
            mcp_tools = mcp_client.list_tools_sync()
            
            # Combine local tools with MCP tools
            all_tools = [
                answer_general_mortgage_questions,
                answer_existing_mortgage_questions,
                calculator
            ] + mcp_tools
            
            # Create supervisor agent
            supervisor = Agent(
                model="us.anthropic.claude-3-5-haiku-20241022-v1:0",
                system_prompt="""
                Your role is to provide a unified experience for all things related to mortgages. You are a supervisor who oversees answering
                customer questions related to general mortgages questions and queries about the existing mortgage.
                
                For general questions, use the answer_general_mortgage_questions tool.
                For questions on existing mortgage, use the answer_existing_mortgage_questions tool.
                If asked for a complicated calculation, use your code interpreter to be sure it's done accurately.
                
                You also have access to MCP tools that can perform additional function to get the credit score of existing customer.
                Use these tools when appropriate for the customer's query.
                
                IMPORTANT: When using credit check tools, return ONLY the credit score value without additional analysis or explanations.
                For other queries, synthesize the details from the response of the tools used into a comprehensive answer provided back to the customer.
                """,
                tools=all_tools
            )
            
            response = supervisor(user_input)
            return str(response)
            
    except Exception as e:
        print(f"Error with MCP integration: {str(e)}")
        
        # Fallback to local tools only
        supervisor = Agent(
            model="us.anthropic.claude-3-5-haiku-20241022-v1:0",
            system_prompt="""
            Your role is to provide a unified experience for all things related to mortgages. You are a supervisor who oversees answering
            customer questions related to general mortgages questions and queries about the existing mortgage.

            For general questions, use the answer_general_mortgage_questions tool.
            For questions on existing mortgage, use the answer_existing_mortgage_questions tool.
            If asked for a complicated calculation, use your code interpreter to be sure it's done accurately.
            
            Synthesize the details from the response of the tools used into a comprehensive answer provided back to the customer.
            """,
            tools=[answer_general_mortgage_questions, answer_existing_mortgage_questions, calculator]
        )
        
        response = supervisor(user_input)
        return str(response)

if __name__ == "__main__":
    app.run()

## 4. Create Requirements File

In [ ]:
%%writefile requirements.txt
mcp>=1.10.0
strands-agents
strands-agents-tools
uv
boto3
bedrock-agentcore
bedrock-agentcore-starter-toolkit

## 6. Deploy to AgentCore Runtime

Configure and deploy the Strands multi-agent system to AgentCore Runtime:

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

boto_session = Session()
region = boto_session.region_name
print(f"Using AWS region: {region}")

agentcore_runtime = Runtime()
agent_name = "strands_multi_agent_mortgage_assistant"

# Environment variables for the deployed agent
env_vars = {
    "MCP_URL": mcp_url,
    "COGNITO_CLIENT_ID": cognito_config['client_id'],
    "COGNITO_USER_POOL_ID": cognito_config['user_pool_id'],
    "COGNITO_USERNAME": cognito_config['username'],
    "COGNITO_PASSWORD": cognito_config['password'],
    "KNOWLEDGE_BASE_ID": kb_id,
    "AWS_REGION" : 'us-east-1'
}

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="strands_multi_agent.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name,
)
print("Configuration completed ✓")
response

## 7. Launch Agent to AgentCore Runtime

In [ ]:
print("Launching Strands Multi-Agent to AgentCore Runtime...")
launch_result = agentcore_runtime.launch(env_vars=env_vars)
print("Launch completed ✓")

## 8. Check Deployment Status

In [ ]:
import time

print("Checking deployment status...")
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']

while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(f"Status: {status}")

print(f"Final Status: {status}")

if status == 'READY':
    print("✓ Strands Multi-Agent successfully deployed to AgentCore Runtime!")
    
    # Store the agent endpoint for future use
    agent_endpoint = launch_result.agent_arn
    %store agent_endpoint
    print(f"Agent ARN: {agent_endpoint}")
else:
    print(f"✗ Deployment failed with status: {status}")

## 9. Test Deployed Agent

In [ ]:
# Test the deployed agent
test_queries = [
    "What is my credit score? My customer ID is 1111",
    "I am customer 3345, when's my next payment due?",
    "Why do so many people choose a 30-year mortgage?"
]

print("Testing deployed Strands Multi-Agent on AgentCore Runtime...\n")

for i, query in enumerate(test_queries, 1):
    print(f"{i}. Query: {query}")
    print("-" * 50)
    
    try:
        invoke_response = agentcore_runtime.invoke({"prompt": query})
        response_text = invoke_response['response'][0]
        print(f"Response: {response_text}\n")
    except Exception as e:
        print(f"Error: {str(e)}\n")
    
    time.sleep(2)

## 10. Test with Boto3 Client

In [ ]:
import boto3
import json

# Test with boto3 client
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client('bedrock-agentcore', region_name=region)

print("Testing with boto3 client...")

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "What is my credit score? Customer ID is 2222"})
)

if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                content.append(line)
    print("\n".join(content))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
        print(json.loads(events[0].decode("utf-8")))
    except Exception as e:
        print(f"Error reading response: {e}")

## 11. Cleanup (Optional)

Clean up the deployed resources:

In [ ]:
# Uncomment to cleanup resources
import boto3

agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=region)
ecr_client = boto3.client('ecr', region_name=region)

# Delete AgentCore Runtime
runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

# Delete ECR repository
response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

print("Cleanup completed ✓")

## Summary

This notebook demonstrates how to:

1. **Create a Strands Multi-Agent System** with mortgage-specific functionality
2. **Deploy to AgentCore Runtime** using the BedrockAgentCoreApp framework
3. **Integrate with MCP Tools** hosted on another AgentCore Runtime
4. **Handle Authentication** between AgentCore Runtime instances
5. **Test the Deployed System** with various mortgage-related queries

### Architecture Benefits:

- **Scalable**: Both agent system and MCP tools run on serverless AgentCore Runtime
- **Secure**: Enterprise-grade security with JWT authentication
- **Maintainable**: Separation of concerns between agent logic and tool services
- **Flexible**: Easy to add new agents or MCP tools independently

The deployed system provides a production-ready mortgage assistant that can handle general questions, existing customer inquiries, and credit checks through the integrated MCP server.